task name: Implement SdcaNonCalibratedWithOptions
task description: Implement the SdcaNonCalibratedWithOptions example using the provided C# code

In [ ]:
#r "nuget:Microsoft.ML,1.5.4"

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;

In [ ]:
var mlContext = new MLContext(seed: 0);

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;

var mlContext = new MLContext(seed: 0);

// Example with label and 50 feature values. A data set is a collection of such examples.
class DataPoint
{
    public bool Label { get; set; }
    [VectorType(50)]
    public float[] Features { get; set; }
}

IEnumerable<DataPoint> GenerateRandomDataPoints(int count, int seed = 0)
{
    var random = new Random(seed);
    float randomFloat() => (float)random.NextDouble();
    for (int i = 0; i < count; i++)
    {
        var label = randomFloat() > 0.5f;
        yield return new DataPoint
        {
            Label = label,
            Features = Enumerable.Repeat(label, 50)
                .Select(x => x ? randomFloat() : randomFloat() + 0.03f).ToArray()
        };
    }
}

var dataPoints = GenerateRandomDataPoints(1000);
var trainingData = mlContext.Data.LoadFromEnumerable(dataPoints);
trainingData = mlContext.Data.Cache(trainingData);

In [ ]:
var options = new SdcaNonCalibratedBinaryTrainer.Options()
{
    LossFunction = new HingeLoss(),
    ConvergenceTolerance = 0.05f,
    MaximumNumberOfIterations = 30,
    PositiveInstanceWeight = 1.2f,
};

var pipeline = mlContext.BinaryClassification.Trainers.SdcaNonCalibrated(options);

In [ ]:
var model = pipeline.Fit(trainingData);

In [ ]:
var testData = mlContext.Data.LoadFromEnumerable(GenerateRandomDataPoints(500, seed: 123));
var transformedTestData = model.Transform(testData);

In [ ]:
using System;

// Class used to capture predictions.
class Prediction
{
    public bool Label { get; set; }
    public bool PredictedLabel { get; set; }
}

var predictions = mlContext.Data.CreateEnumerable<Prediction>(transformedTestData, reuseRowObject: false).ToList();
foreach (var p in predictions.Take(5))
    Console.WriteLine($"Label: {p.Label}, Prediction: {p.PredictedLabel}");

In [ ]:
var metrics = mlContext.BinaryClassification.EvaluateNonCalibrated(transformedTestData);
PrintMetrics(metrics);

void PrintMetrics(BinaryClassificationMetrics metrics)
{
    Console.WriteLine($"Accuracy: {metrics.Accuracy:F2}");
    Console.WriteLine($"AUC: {metrics.AreaUnderRocCurve:F2}");
    Console.WriteLine($"F1 Score: {metrics.F1Score:F2}");
    Console.WriteLine($"Negative Precision: {metrics.NegativePrecision:F2}");
    Console.WriteLine($"Negative Recall: {metrics.NegativeRecall:F2}");
    Console.WriteLine($"Positive Precision: {metrics.PositivePrecision:F2}");
    Console.WriteLine($"Positive Recall: {metrics.PositiveRecall:F2}");
    Console.WriteLine(metrics.ConfusionMatrix.GetFormattedConfusionTable());
}